In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))  # 一个具有单隐藏层的多层感知机
X = torch.rand(size=(2, 4))
net(X)

tensor([[ 0.0559],
        [-0.0196]], grad_fn=<AddmmBackward0>)

参数访问

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0005,  0.0956,  0.1259,  0.0233, -0.2487,  0.0141,  0.1754, -0.1441]])), ('bias', tensor([-0.2201]))])


In [3]:
# 提取第二个全连接层的偏置
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2201], requires_grad=True)
tensor([-0.2201])


In [4]:
net[2].weight.grad == None

True

In [5]:
# 一次性访问所有参数

# 访问第一个全连接层
print([(name, param.shape) for name, param in net[0].named_parameters()])
# *将列表[(name1, shape1), (name2, shape2), ...]解包为多个独立的参数(name1, shape1), (name2, shape2),...
# print, 打印每个独立的参数(默认空格分隔)
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

# 访问所有层的参数
print(*[(name, param.shape) for name, param in net.named_parameters()])


[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
net.state_dict()['2.bias'].data

tensor([-0.2201])

从嵌套块收集参数

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1651],
        [-0.1648]], grad_fn=<AddmmBackward0>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [9]:
rgnet[0][1][0].bias.data

tensor([-0.1870,  0.3252, -0.0385,  0.4358, -0.4547, -0.2373, -0.0912, -0.4666])

参数初始化

In [ ]:
def init_normal(m):
    """
    权重初始化, 服从高斯分布, 平均值0, 标准差0.01
    偏置初始化为0
    """
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0112,  0.0111,  0.0010, -0.0087]), tensor(0.))

In [ ]:
def init_constant(m):
    """
    权重初始化, 全为1
    偏置初始化为0
    """
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [ ]:
# 对不同层应用不同的初始化

def init_xavier(m):
    """权重初始化服从xavier分布"""
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    """权重初始化全为42"""
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5698, -0.3967, -0.0168,  0.3741])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [14]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])

        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5  # 只保留绝对值>=5的数

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 9.8608, -0.0000,  9.6112,  9.1499],
        [-0.0000, -9.2948,  6.5209,  0.0000]], grad_fn=<SliceBackward0>)

In [ ]:
# 可以手动设置参数
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, 10.6112, 10.1499])

参数绑定

In [20]:
# 我们需要给共享层提供一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
